In [1]:
import numpy as np, pandas as pd, scipy.stats as ss, seaborn as sns, os
from sklearn.decomposition import PCA
from matplotlib import pyplot as plt
%matplotlib inline

def allelic_ANOVA(site, pheno):
    """This regression is equivalent to one-way ANOVA with 2 groups. Return F-statistic.

    Assumes site is coded as -1, 1
    """
    #pheno = pheno.dropna();assert len(pheno) > 0;
    coding = np.array(site,np.float)
    pheno = np.array(pheno,np.float)

    meany = np.mean(pheno)
    meandummy = np.mean(coding)
    ctry = pheno - meany
    ctrdummy = coding - meandummy
    
    # regression coefficient and intercept
    b = np.dot(ctry, ctrdummy)/np.dot(ctrdummy, ctrdummy)
    intercept = meany - b * meandummy
    
    yhat = b * ctrdummy
    len_yhat = np.sqrt(np.dot(yhat,yhat))
    len_y = np.sqrt(np.dot(ctry,ctry))
    df_yhat = 1
    
    error = ctry  - yhat
    len_error = np.sqrt(np.dot(error,error))
    if abs(len_error**2) < 1e-5:
        raise Exception("Zero length error in ANOVA")
    df_error = len(pheno) - 2
    
    # coefficient of determination is R**2
    #R = len_yhat/len_y
    
    # F-statistic
    F = (len_yhat**2/df_yhat) / (len_error**2/df_error)
    return F

def association_logPval(site, pheno):
    F = allelic_ANOVA(site, pheno)
    logP = np.log10(ss.f.sf(F, 1, len(pheno)-2))
    return -logP

def myminmax(x):
    x = np.array(x)
    return (x-x.min())/(x.max()-x.min())

In [2]:
from matplotlib import rc
xl280al = 'XL280'+r'$\alpha$'#+'SS'
xl280_label = 'XL280' +r'$\bf{a}$'
_431_label = '431'+r'$\mathrm{\alpha}$'

In [3]:
## Bring in genotype data
geno_path = '../DATA/CDx-ill-SNP-INDEL-df-104-blocked.csv.gz'
geno_df = pd.read_csv(geno_path)

## Define chromosomes
chrlist = geno_df.Chrom.unique()
## Gather the length of each chromosome
clens = [geno_df[(geno_df.Chrom==chrom)]['Pos'].max() 
         for chrom in chrlist]
## Set mid points
chrom_midpts = 0.5 * np.sum(np.array([a 
                    for a in zip([0]+list(np.cumsum(clens)),
                            np.cumsum(clens))]),axis=1)
## calculate cumulative positions
cumpos = [0] + list(np.cumsum(clens))[:-1]

In [5]:
## Amphotericin B and thermal tolerance
## Set phenotype data path
file_path = '../DATA/Tecan_qtlruns_Jan162019_curated_baselined_median51.csv.gz'
## Bring in phenotype data
df = pd.read_csv(file_path,index_col=0)
dfcol = df.columns.tolist()
time_ix = dfcol.index('6300s')
times_ixs = dfcol[time_ix:]
#df.index = df.strain
df = df[(df.strain.isin(geno_df.columns)) & (df.temp!=38)]
timex = [np.round(int(a[:-1])/(60**2),3) for a in times_ixs]
dx = np.max(np.diff(timex))
df.index = df.strain
df.drop('strain',inplace=True,axis=1)
df['AUC'] = df[times_ixs].sum(axis=1)*dx
df.head()

,date,row,column,plate,temp,amphB,length,Instrument,saturatedOD,premedia,...,250201s,251100s,252000s,252901s,253800s,254700s,255601s,256500s,257400s,AUC
strain,,,,,,,,,,,,,,,,,,,,,
SS-A837,Sep182017,e,7,2,30,0.0,72,Loaner,1.4500,YPD,...,1.4098,1.4098,1.4098,1.4098,1.4098,1.4098,1.4098,1.4098,1.4098,74.95100
SS-A837,Nov12017,e,7,2,30,0.0,72,Moon,1.4392,YPD,...,1.4162,1.4172,1.4172,1.4172,1.4172,1.4172,1.4172,1.4182,1.4182,70.85075
SS-A837,Nov82017,e,7,2,30,0.0,72,Moon,1.4800,YPD,...,1.4158,1.4168,1.4168,1.4168,1.4178,1.4178,1.4188,1.4188,1.4188,70.48700
SS-A837,Feb212018,e,7,2,30,0.0,72,Moon,1.6927,YPD,...,1.4262,1.4262,1.4272,1.4272,1.4272,1.4272,1.4272,1.4272,1.4272,74.84250
SS-B307,Sep182017,b,2,2,30,0.0,72,Loaner,1.5400,YPD,...,0.9590,0.9590,0.9590,0.9590,0.9610,0.9610,0.9620,0.9620,0.9620,43.06800


In [ ]:
auc = df.groupby(['temp','amphB','strain']).agg(np.median
        ).copy().reset_index()[['temp','amphB','strain','AUC']]
auc.index = auc.strain
auc.drop('strain',inplace=True,axis=1)

aucs = pd.concat([a[1]['AUC'] for a in auc.groupby(['temp','amphB'])],axis=1)
aucs.columns = ['_'.join([str(k) for k in a[0]]) for a in auc.groupby(['temp','amphB'])]
print(aucs.shape[0])
aucs.head()

In [9]:
## Capsule
cap_path = '../DATA/Capsule_Phenotypes.csv'
capsule = pd.read_csv(cap_path)
capsule.drop('Sample.1',inplace=True,axis=1)
capsule.columns = ['strain'] + capsule.columns.tolist()[1:]
print(capsule.shape[0])
capsule.index = capsule.strain
capsule.drop('strain',inplace=True,axis=1)
capsule.head()

104


,Date,Cell_and_Capsule,Cell,Capsule,Cell_and_Capsule_diameter,Cell_diameter,Capsule_diameter,Circles,Residual,Cell_Residual,QTL
strain,,,,,,,,,,,
SS-A837,7-120,2954.0,1093.0,1935.5,61.323907,37.304177,25.117309,15,2.144624,-1.179139,1.0
SS-B307,7-120,3015.0,831.0,2065.5,61.958189,32.512533,26.994920,15,3.635013,-6.198297,1.0
SS-B310,7-120,6371.0,2168.0,3964.5,90.064104,52.517844,34.967517,15,-5.550718,3.725538,0.0
SS-B311,7-150,3293.5,1023.5,2096.5,64.756295,36.099316,26.319158,15,1.251042,-3.615184,1.0
SS-B312,11-22,7008.5,1610.0,5398.5,94.452263,45.269625,47.179251,15,3.982097,-5.096695,0.0


In [13]:
## Melanization
melanin = pd.read_csv('../DATA/Melanin_Phenotypes.csv',index_col=0)
print(melanin.shape[0])
melanin.head()

104


,QTL,Melanin
strain,,
SS-B357,0.0,17455.744000
SS-B360,1.0,13758.783333
SS-B364,0.0,5925.816000
SS-B366,0.0,17842.346000
SS-B368,0.0,19223.485333


In [ ]:
## Fludioxonil
fludio = pd.read_csv('~/Cdx-predict-fludio-phenotype.csv')
fludio.columns = ['strain'] + fludio.columns.tolist()[1:]
to_drop = fludio[(fludio.strain=='SS-B575')].index[0]
fludio.drop(to_drop,axis=0,inplace=True)
fludio.index = fludio.strain
fludio.drop('strain',inplace=True,axis=1)
fludio['Outcome'] = fludio['Outcome'].apply(float)
print(fludio.shape[0])
fludio.head()

In [ ]:
## Hydrogenperoxide
H2o2 = pd.read_csv('/home/croth/H2O2_phenotypes.csv')
H2o2.columns = ['strain'] + H2o2.columns.tolist()[1:]
H2o2.index = H2o2.strain
H2o2.drop('strain',axis=1,inplace=True)
H2o2.head()

In [ ]:
segs = [s for s in sorted(H2o2.index.tolist()) if s in geno_df.columns]
len(segs)

In [ ]:
segs[:5]

In [ ]:
capsule.T[segs].T['Residual'].values.shape

In [ ]:
pheno = pd.DataFrame(index=segs)
pheno['Melanin'] = melanin.T[segs].T['Melanin'].values
#pheno['Annulus'] = np.array(capsule.T[segs].T['Residual'].values,dtype=float)
pheno['Fludioxonil'] = fludio.T[segs].T['Outcome'].apply(float)
pheno['H2O2'] = H2o2.T[segs].T['Score']

for k in aucs.columns:
    pheno[k] = aucs[k].T[pheno.index]

pheno.shape

from sklearn import preprocessing

minmax = preprocessing.MinMaxScaler()

def myminmax(x):
    return minmax.fit_transform(x.values.reshape(-1, 1)).T[0]

In [ ]:
loci = geno_df.Block.drop_duplicates().index.tolist()
geno = geno_df.T[loci].T[pheno.index]
for s in geno.columns:
    geno[s] = np.array(geno[s],dtype=float)
geno.head()

In [ ]:
melanin_qtl = pd.DataFrame(geno[pheno.index].apply(
                association_logPval,axis=1,
                args=[pheno['Melanin'].values]))
melanin_qtl.columns = ['Melanin']
melanin_qtl[['Pos','Chrom']] = geno_df[['Pos','Chrom']].T[loci].T
melanin_qtl.head()

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(12,3))
for i,c in enumerate(chrlist):
    tp = melanin_qtl[(melanin_qtl.Chrom==c)]
    plt.plot(tp.Pos.values+cumpos[i],tp.Melanin.values,'.')
plt.xticks(chrom_midpts,np.arange(1,15));
plt.xlabel('Chromosome',fontsize=14)
plt.ylabel('-log$_{10}$ (p-value)',fontsize=14);

In [ ]:
pheno['QTL14'] = geno.T[melanin_qtl.Melanin.idxmax()][pheno.index]

In [ ]:
pheno.head()

In [ ]:
sns.scatterplot(pheno['37_0.0'], pheno['39_0.0'], 
                hue='QTL14', data=pheno, 
                alpha=0.5, legend=None);

In [ ]:
osmo = pd.read_csv('./Cdx_osmo_NaCl_1M.csv')
osmo.index = osmo.strain
osmo.head()

In [ ]:
pheno['NaCl_1M'] = osmo['mean_intensity'].T[pheno.index]

In [ ]:
capsule.columns = capsule.columns[:-3].tolist() + ['Capsule_diameter_residual','Cell_diameter_residual','QTL']

In [ ]:
cap_phenos = [a for a in capsule.columns.tolist() if a not in ['Date','Circles','QTL']]

In [ ]:
cap_phenos

In [ ]:
for p in cap_phenos:
    pheno[p] = np.array(capsule.T[pheno.index].T[p].values,dtype=float)

In [ ]:
for p in cap_phenos:
    print(p,ss.pearsonr(*pheno[['Melanin',p]].T.values),'\n')

In [ ]:
for p in cap_phenos:
    print(p,ss.pearsonr(*pheno[['39_0.0',p]].T.values),'\n')

In [ ]:
pheno_to_analysis = [a for a in pheno.columns.tolist() 
                     if a not in ['QTL14','Cell_and_Capsule','Cell',
                                  'Capsule','Cell_diameter','Cell_diameter_residual',
                                 'SSK1','SSK2','RIC8']]

In [ ]:
pheno_to_analysis

In [ ]:
pheno_new_names = []
for j in pheno_to_analysis:
    if len(j.split('_')) == 2 and len(j.split('_')[0]) == 2:
        
        if float(j.split('_')[1]) == 0.0:
            k = '0.000'
        else:
            k = j.split('_')[1]
        
        new_name = j.split('_')[0]+'°C & ' + k+u' \u03BCg/ml'
    
    elif j == 'H2O2':
        new_name = 'H$_{2}$O$_{2}$'
    elif j == 'Melanin':
        new_name = 'Melanization'
    elif j == 'Fludioxonil':
        new_name = j + ' 100 \u03BCg/ml'
    elif j[0] == 'C':
        new_name = ' '.join(j.split('_')) 
        new_name = new_name.lower().capitalize()
        if 'diameter' not in new_name:
            new_name = new_name.lower()+' area'
    elif j[0] == 'N' and len(j.split('_')) == 2:
        new_name = ' '.join(j.split('_'))
    else:
        new_name = j.lower().capitalize()
    
    pheno_new_names.append(new_name)

In [ ]:
for i,k in enumerate(pheno_new_names):
    
    print(pheno_to_analysis[i],k)

In [ ]:
pheno['Melanin'] = -pheno['Melanin']

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(8,6))
sns.heatmap( pheno[pheno_to_analysis].corr(method='spearman'),
            vmin = -1, vmax=1,cmap='cividis_r');
fs = 12
xticks = plt.xticks()[0]
plt.xticks(xticks,pheno_new_names,fontsize=fs);
yticks = plt.yticks()[0]
plt.yticks(yticks,pheno_new_names,fontsize=fs);

x1 = -0.15
fig.add_axes([x1,0.29,0.0001,.46])
plt.xticks([]);plt.yticks([]);
plt.ylabel('Temperature & amphotericin B',fontsize=fs)
fig.add_axes([x1,0.29,0.04,0.0001])
plt.xticks([]);plt.yticks([]);
fig.add_axes([x1,.75,0.04,0.0001])
plt.xticks([]);plt.yticks([]);

y1 = -0.23
x2 = 0.225
fig.add_axes([x2,y1,0.38,.00001])
plt.xticks([]);plt.yticks([]);
plt.xlabel('Temperature & amphotericin B',fontsize=fs)
fig.add_axes([x2,y1,0.0001,0.04])
plt.xticks([]);plt.yticks([]);
fig.add_axes([x2+.38,y1,0.0001,0.04])
plt.xticks([]);plt.yticks([]);

#plt.savefig('./Phenotypes_corrilation_plot.png',
plt.savefig('/home/croth/QTL_FIGS/SUPP/FIGS/S8_Fig.pdf',
                dpi=300,bbox_inches='tight');

In [ ]:
for_pm = np.array(pheno_to_analysis)[[0,2,11,-1]]

In [ ]:
pm_new_names = np.array(pheno_new_names)[[0,2,11,-1]]

In [ ]:
#pheno['Melanin'] = -1*pheno['Melanin']

In [ ]:
pheno[for_pm].corr(method='spearman')

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(8,6))
sns.heatmap( pheno[for_pm].corr(method='spearman'),
            vmin = -1, vmax=1,cmap='cividis_r',annot=True);
fs = 12
xticks = plt.xticks()[0]
plt.xticks(xticks,pm_new_names,fontsize=fs,rotation=90);
yticks = plt.yticks()[0]
plt.yticks(yticks,pm_new_names,fontsize=fs,rotation=0);
plt.savefig('./Phenotypes_corrilation_for_PM.png',
                dpi=200,bbox_inches='tight');

In [ ]:
assert 1 == 0

In [ ]:
from mpl_toolkits import mplot3d

In [ ]:
qtlloci = pd.read_csv('~/Desktop/Cdx_QTL_SNPs.csv',index_col=0)

In [ ]:
pheno['RIC8'] = qtlloci[(qtlloci.Chrom=='Chr14')][pheno.index].values[0]
pheno['SSK1'] = qtlloci[(qtlloci.Chrom=='Chr02')][pheno.index].values[0]
pheno['SSK2'] = qtlloci[(qtlloci.Chrom=='Chr12')][pheno.index].values[0]

In [ ]:
pheno.tail()

In [ ]:
pheno['new_mosaic'] = -1

for c,(i,a) in enumerate(pheno.groupby(['SSK2','RIC8'])):
    
    pheno.loc[a.index,'new_mosaic'] = c

In [ ]:
assert pheno.new_mosaic.min()!=-1

In [ ]:
new_mosaic = pheno[['RIC8','SSK2','new_mosaic']].drop_duplicates().sort_values('new_mosaic').reset_index(drop=True).copy()
new_mosaic

In [ ]:
new_xlabel = '$SSK2$: %s\n$RIC8$: %s'
parental_label = [xl280_label,_431_label]
new_xlabels = []

for i,j in new_mosaic.iterrows():
    
    new_xlabels.append(new_xlabel%(parental_label[int(j.SSK2)],parental_label[int(j.RIC8)]))
new_xlabels

In [ ]:
pheno['Melanin'] = pheno['Melanin'].apply(np.abs)
pheno['SSK1r'] = pheno['SSK1'].replace(dict(zip([0,1],[xl280_label,_431_label])))

In [ ]:
sns.boxplot(x='new_mosaic',y='Melanin',data=pheno,color='w',showfliers = False)
sns.swarmplot(x='new_mosaic',y='Melanin',hue='SSK1r',
              data=pheno,hue_order=[xl280_label,_431_label]);
plt.legend(title='$SSK1$',bbox_to_anchor=(1.3,.85),fontsize=10)
plt.xlabel('\n$SSK2$ and $RIC8$ genotypes',fontsize=14)
plt.xticks(np.arange(0,4),new_xlabels,fontsize=10);
plt.ylabel('Mean Intensity\n$\longleftarrow$Dark - Light$\longrightarrow$',fontsize=14);

for i,j in pheno.groupby(['new_mosaic','SSK1']):
    
    plt.plot(i[0]+0.5,j.Melanin.median(),'^',ms =10,
             fillstyle='none',color='tab:blue' if i[1] == 0 else 'tab:orange')
plt.xlim(-0.5,3.7);
plt.savefig('/home/croth/Desktop/Melanization_RIC8:SSK2_by_SSK1.png',
            dpi=100,bbox_inches='tight');

In [ ]:
sns.boxplot(x='new_mosaic',y='Capsule_diameter',data=pheno,color='w',showfliers = False)
sns.swarmplot(x='new_mosaic',y='Capsule_diameter',hue='SSK1r',
              data=pheno,hue_order=[xl280_label,_431_label]);
plt.legend(title='$SSK1$',bbox_to_anchor=(1.3,.85),fontsize=10)
plt.xlabel('\n$SSK2$ and $RIC8$ genotypes',fontsize=14)
plt.xticks(np.arange(0,4),new_xlabels,fontsize=10);
plt.ylabel('Capsule diameter residual',fontsize=14);

for i,j in pheno.groupby(['new_mosaic','SSK1']):
    
    plt.plot(i[0]+0.5,j.Capsule_diameter.median(),'^',ms =10,
             fillstyle='none',color='tab:blue' if i[1] == 0 else 'tab:orange')
plt.xlim(-0.5,3.7);
plt.savefig('/home/croth/Desktop/Capsule_diameter_RIC8:SSK2_by_SSK1.png',
            dpi=100,bbox_inches='tight');

In [ ]:
sns.boxplot(x='new_mosaic',y='Capsule_diameter_residual',data=pheno,color='w',showfliers = False)
sns.swarmplot(x='new_mosaic',y='Capsule_diameter_residual',hue='SSK1r',
              data=pheno,hue_order=[xl280_label,_431_label]);
plt.legend(title='$SSK1$',bbox_to_anchor=(1.3,.85),fontsize=10)
plt.xlabel('\n$SSK2$ and $RIC8$ genotypes',fontsize=14)
plt.xticks(np.arange(0,4),new_xlabels,fontsize=10);
plt.ylabel('Capsule diameter residual',fontsize=14);

for i,j in pheno.groupby(['new_mosaic','SSK1']):
    
    plt.plot(i[0]+0.5,j.Capsule_diameter_residual.median(),'^',ms =10,
             fillstyle='none',color='tab:blue' if i[1] == 0 else 'tab:orange')
plt.xlim(-0.5,3.7);
plt.savefig('/home/croth/Desktop/Capsule_diameter_residual_RIC8:SSK2_by_SSK1.png',
            dpi=100,bbox_inches='tight');

In [ ]:
sns.boxplot(x='new_mosaic',y='Cell_diameter',data=pheno,color='w',showfliers = False)
sns.swarmplot(x='new_mosaic',y='Cell_diameter',hue='SSK1r',
              data=pheno,hue_order=[xl280_label,_431_label]);
plt.legend(title='$SSK1$',bbox_to_anchor=(1.3,.85),fontsize=10)
plt.xlabel('\n$SSK2$ and $RIC8$ genotypes',fontsize=14)
plt.xticks(np.arange(0,4),new_xlabels,fontsize=10);
plt.ylabel('Cell diameter residual',fontsize=14);

for i,j in pheno.groupby(['new_mosaic','SSK1']):
    
    plt.plot(i[0]+0.5,j.Cell_diameter.median(),'^',ms =10,
             fillstyle='none',color='tab:blue' if i[1] == 0 else 'tab:orange')
plt.xlim(-0.5,3.7);
plt.savefig('/home/croth/Desktop/Cell_diameter_RIC8:SSK2_by_SSK1.png',
            dpi=100,bbox_inches='tight');

In [ ]:
sns.boxplot(x='new_mosaic',y='Cell_diameter_residual',data=pheno,color='w',showfliers = False)
sns.swarmplot(x='new_mosaic',y='Cell_diameter_residual',hue='SSK1r',
              data=pheno,hue_order=[xl280_label,_431_label]);
plt.legend(title='$SSK1$',bbox_to_anchor=(1.3,.85),fontsize=10)
plt.xlabel('\n$SSK2$ and $RIC8$ genotypes',fontsize=14)
plt.xticks(np.arange(0,4),new_xlabels,fontsize=10);
plt.ylabel('Cell diameter residual',fontsize=14);

for i,j in pheno.groupby(['new_mosaic','SSK1']):
    
    plt.plot(i[0]+0.5,j.Cell_diameter_residual.median(),'^',ms =10,
             fillstyle='none',color='tab:blue' if i[1] == 0 else 'tab:orange')
plt.xlim(-0.5,3.7);
plt.savefig('/home/croth/Desktop/Cell_diameter_residual_RIC8:SSK2_by_SSK1.png',
            dpi=100,bbox_inches='tight');

In [ ]:
pheno.head()

In [ ]:
sns.swarmplot(x='SSK1',y='H2O2',data=pheno);
plt.plot(np.array([0,1])+.25, pheno.groupby('SSK1')['H2O2'].mean().values,'k^',fillstyle='none',ms=10);
plt.savefig('/home/croth/Desktop/H2O2_bySSK1.png',dpi=100,bbox_inches='tight');


In [ ]:
pheno.groupby('SSK1')['H2O2'].mean()

In [ ]:
pheno['SSK2r'] = pheno['SSK2'].replace(dict(zip([0,1],[xl280_label,_431_label])))
pheno['RIC8r'] = pheno['RIC8'].replace(dict(zip([0,1],[xl280_label,_431_label])))

In [ ]:
for i,a in enumerate(pheno.groupby(['SSK1','SSK2'])):
    
    pheno.loc[a[1].index,'new_mosaic'] = i

In [ ]:
pheno[['SSK1','SSK2','new_mosaic']].drop_duplicates().sort_values(['SSK1','SSK2'])

In [ ]:
myphenooi = 'H2O2'
myylabel = 'H$_{2}$O$_{2}$ Growth Score'
fig, ax = plt.subplots(3,1,figsize=(16,16),sharex=False,sharey=True)

for ri,px in enumerate([('SSK1','SSK2','RIC8'),('SSK2','RIC8','SSK1'),('SSK1','RIC8','SSK2')]):
    
    g0 = px[0]
    g1 = px[1]
    g2 = px[2]

    pheno['new_mosaic'] = -1
    for i,a in enumerate(pheno.groupby([g0,g1])):
        pheno.loc[a[1].index,'new_mosaic'] = i
    
    temp_mosaic = pheno[(pheno[g0]==1) & (pheno[g1]==1)].new_mosaic.min()
    pheno.loc[(pheno[g0]==1) & (pheno[g1]==1),'new_mosaic'] = -temp_mosaic
    pheno.loc[(pheno.new_mosaic==1),'new_mosaic'] = temp_mosaic
    pheno.loc[(pheno.new_mosaic==-temp_mosaic),'new_mosaic'] = 1
    assert pheno.new_mosaic.min()>=0
    new_mosaic_df = pheno[[g0,g1,'new_mosaic']].drop_duplicates().sort_values('new_mosaic').copy()

    plt.sca(ax[ri])
    sns.swarmplot(x='new_mosaic',y=myphenooi,data=pheno,hue='%sr'%g2,
              hue_order=[xl280_label,_431_label]);
    lg = plt.legend(bbox_to_anchor=(0.5,0.68),title='$%s$'%g2,fontsize=14);
    title = lg.get_title()
    title.set_fontsize(14)
    plt.ylabel(myylabel,fontsize=14)

    mean_scores = pheno.groupby(['new_mosaic',g2]).mean()
    for i,j in mean_scores.iterrows():
        plt.plot(i[0]+0.2,j[myphenooi],'^',ms = 15,fillstyle='none',
             color='tab:blue' if i[1] == 0 else 'tab:orange',alpha=0.69);

    xlabel = '$%s$: %s\n$%s$: %s'
    new_x_labels = []

    for i,j in new_mosaic_df.iterrows():
    
        xlab = xlabel%(g0,parental_label[int(j[g0])],g1,parental_label[int(j[g1])])
        new_x_labels.append(xlab)
    
    plt.xticks(np.arange(0,4),new_x_labels,fontsize=14);
    plt.xlabel(' ')
plt.savefig('/home/croth/Desktop/Parametric/%s_bygeno.png'%myphenooi,
            bbox_inches='tight',dpi=200);

In [ ]:
pheno['new_mosaic'] = -1
for i,a in enumerate(pheno.groupby(['SSK1','SSK2','RIC8'])):
    
    pheno.loc[a[1].index,'new_mosaic'] = i
    
temp_mosaic = pheno[(pheno.RIC8==1) & (pheno.SSK2==1) & (pheno.SSK1==1)].new_mosaic.min()

pheno.loc[(pheno.RIC8==1) & (pheno.SSK2==1) & (pheno.SSK1==1),'new_mosaic'] = -temp_mosaic

pheno.loc[(pheno.new_mosaic==1),'new_mosaic'] = temp_mosaic
pheno.loc[(pheno.new_mosaic==-temp_mosaic),'new_mosaic'] = 1

assert pheno.new_mosaic.min()>=0

new_mosaic_df = pheno[['SSK1','SSK2','RIC8','new_mosaic']].drop_duplicates().sort_values('new_mosaic').copy()

xlabel = '$SSK1$: %s\n$SSK2$: %s\n$RIC8$: %s'
new_x_labels = []

for i,j in new_mosaic_df.iterrows():
    
    xlab = xlabel%(parental_label[int(j.SSK1)],parental_label[int(j.SSK2)],parental_label[int(j.RIC8)])
    new_x_labels.append(xlab)
    
fig,ax= plt.subplots(1,1,figsize=(12,3))
#sns.boxplot(x='new_mosaic',y='H2O2',data=pheno,color='w')
sns.swarmplot(x='new_mosaic',y='H2O2',data=pheno);

for i,j in pheno.groupby('new_mosaic'):
    
    plt.plot(i+.25,j.H2O2.median(),'^',fillstyle='none',ms=10)

plt.ylabel('H$_{2}$O$_{2}$ Growth Score')
plt.xlabel('QTG genotypes')
plt.xticks(new_mosaic_df.new_mosaic,new_x_labels,fontsize=10);
plt.savefig('/home/croth/Desktop/H2O2_all_combos.png',dpi=100,bbox_inches='tight');

In [ ]:
pheno.columns

In [ ]:
pheno[['H2O2','NaCl_1M','SSK1','SSK2','RIC8']].to_csv('/home/croth/Desktop/H2O2_data.csv',index=True);

In [ ]:
myphenooi = 'Melanin'
myylabel = 'Mean Intensity\n$\longleftarrow$Dark - Light$\longrightarrow$'#'H$_{2}$O$_{2}$ Growth Score'
fig, ax = plt.subplots(3,1,figsize=(16,16),sharex=False,sharey=True)

for ri,px in enumerate([('SSK1','SSK2','RIC8'),('SSK2','RIC8','SSK1'),('SSK1','RIC8','SSK2')]):
    
    g0 = px[0]
    g1 = px[1]
    g2 = px[2]

    pheno['new_mosaic'] = -1
    for i,a in enumerate(pheno.groupby([g0,g1])):
        pheno.loc[a[1].index,'new_mosaic'] = i
    
    temp_mosaic = pheno[(pheno[g0]==1) & (pheno[g1]==1)].new_mosaic.min()
    pheno.loc[(pheno[g0]==1) & (pheno[g1]==1),'new_mosaic'] = -temp_mosaic
    pheno.loc[(pheno.new_mosaic==1),'new_mosaic'] = temp_mosaic
    pheno.loc[(pheno.new_mosaic==-temp_mosaic),'new_mosaic'] = 1
    assert pheno.new_mosaic.min()>=0
    new_mosaic_df = pheno[[g0,g1,'new_mosaic']].drop_duplicates().sort_values('new_mosaic').copy()

    plt.sca(ax[ri])
    sns.swarmplot(x='new_mosaic',y=myphenooi,data=pheno,hue='%sr'%g2,
              hue_order=[xl280_label,_431_label]);
    lg = plt.legend(bbox_to_anchor=(0.5,0.68),title='$%s$'%g2,fontsize=14);
    title = lg.get_title()
    title.set_fontsize(14)
    plt.ylabel(myylabel,fontsize=14)

    mean_scores = pheno.groupby(['new_mosaic',g2]).mean()
    for i,j in mean_scores.iterrows():
        plt.plot(i[0]+0.2,j[myphenooi],'^',ms = 15,fillstyle='none',
             color='tab:blue' if i[1] == 0 else 'tab:orange',alpha=0.69);

    xlabel = '$%s$: %s\n$%s$: %s'
    new_x_labels = []

    for i,j in new_mosaic_df.iterrows():
    
        xlab = xlabel%(g0,parental_label[int(j[g0])],g1,parental_label[int(j[g1])])
        new_x_labels.append(xlab)
    
    plt.xticks(np.arange(0,4),new_x_labels,fontsize=14);
    plt.xlabel(' ')
plt.savefig('/home/croth/Desktop/Parametric/%s_bygeno.png'%myphenooi,
            bbox_inches='tight',dpi=200);

In [ ]:
myphenooi = '37_0.125'
myylabel = 'AUC at 37 with 0.125'
fig, ax = plt.subplots(3,1,figsize=(16,16),sharex=False,sharey=True)

for ri,px in enumerate([('SSK1','SSK2','RIC8'),('SSK2','RIC8','SSK1'),('SSK1','RIC8','SSK2')]):
    
    g0 = px[0]
    g1 = px[1]
    g2 = px[2]

    pheno['new_mosaic'] = -1
    for i,a in enumerate(pheno.groupby([g0,g1])):
        pheno.loc[a[1].index,'new_mosaic'] = i
    
    temp_mosaic = pheno[(pheno[g0]==1) & (pheno[g1]==1)].new_mosaic.min()
    pheno.loc[(pheno[g0]==1) & (pheno[g1]==1),'new_mosaic'] = -temp_mosaic
    pheno.loc[(pheno.new_mosaic==1),'new_mosaic'] = temp_mosaic
    pheno.loc[(pheno.new_mosaic==-temp_mosaic),'new_mosaic'] = 1
    assert pheno.new_mosaic.min()>=0
    new_mosaic_df = pheno[[g0,g1,'new_mosaic']].drop_duplicates().sort_values('new_mosaic').copy()

    plt.sca(ax[ri])
    sns.swarmplot(x='new_mosaic',y=myphenooi,data=pheno,hue='%sr'%g2,
              hue_order=[xl280_label,_431_label]);
    lg = plt.legend(bbox_to_anchor=(0.5,0.68),title='$%s$'%g2,fontsize=14);
    title = lg.get_title()
    title.set_fontsize(14)
    plt.ylabel(myylabel,fontsize=14)

    mean_scores = pheno.groupby(['new_mosaic',g2]).mean()
    for i,j in mean_scores.iterrows():
        plt.plot(i[0]+0.2,j[myphenooi],'^',ms = 15,fillstyle='none',
             color='tab:blue' if i[1] == 0 else 'tab:orange',alpha=0.69);

    xlabel = '$%s$: %s\n$%s$: %s'
    new_x_labels = []

    for i,j in new_mosaic_df.iterrows():
    
        xlab = xlabel%(g0,parental_label[int(j[g0])],g1,parental_label[int(j[g1])])
        new_x_labels.append(xlab)
    
    plt.xticks(np.arange(0,4),new_x_labels,fontsize=14);
    plt.xlabel(' ')
plt.savefig('/home/croth/Desktop/Parametric/%s_bygeno.png'%myphenooi,
            bbox_inches='tight',dpi=200);

In [ ]:
pheno.columns

In [ ]:
pheno['new_mosaic'] = -1
for i,a in enumerate(pheno.groupby(['SSK1','SSK2','RIC8'])):
    
    pheno.loc[a[1].index,'new_mosaic'] = i
    
temp_mosaic = pheno[(pheno.RIC8==1) & (pheno.SSK2==1) & (pheno.SSK1==1)].new_mosaic.min()

pheno.loc[(pheno.RIC8==1) & (pheno.SSK2==1) & (pheno.SSK1==1),'new_mosaic'] = -temp_mosaic

pheno.loc[(pheno.new_mosaic==1),'new_mosaic'] = temp_mosaic
pheno.loc[(pheno.new_mosaic==-temp_mosaic),'new_mosaic'] = 1

assert pheno.new_mosaic.min()>=0

new_mosaic_df = pheno[['SSK1','SSK2','RIC8','new_mosaic']].drop_duplicates().sort_values('new_mosaic').copy()

xlabel = '$SSK1$: %s\n$SSK2$: %s\n$RIC8$: %s'
new_x_labels = []

for i,j in new_mosaic_df.iterrows():
    
    xlab = xlabel%(parental_label[int(j.SSK1)],parental_label[int(j.SSK2)],parental_label[int(j.RIC8)])
    new_x_labels.append(xlab)
    

growth_curves = ['37_%s'%d for d in [0.0,0.075,0.125,0.175]]

fig,ax= plt.subplots(len(growth_curves),1,figsize=(12,10),sharex=True)
#sns.boxplot(x='new_mosaic',y='H2O2',data=pheno,color='w')
for ix,pt in enumerate(growth_curves):
    plt.sca(ax[ix])
    sns.swarmplot(x='new_mosaic',y=pt, data=pheno);

    for i,j in pheno.groupby('new_mosaic'):
    
        plt.plot(i+.25,j[pt].median(),'^',fillstyle='none',ms=10)

    plt.ylabel('AUC at %s\n with %s amph. B'%(pt.split('_')[0],pt.split('_')[1]),fontsize=12)
    plt.xlabel('')
plt.xlabel('QTG genotypes')
plt.xticks(new_mosaic_df.new_mosaic,new_x_labels,fontsize=10);
plt.savefig('/home/croth/Desktop/Drug_curves_37C_all_combos.png',
            dpi=100,bbox_inches='tight');

In [ ]:
pheno['new_mosaic'] = -1
for i,a in enumerate(pheno.groupby(['SSK1','SSK2','RIC8'])):
    
    pheno.loc[a[1].index,'new_mosaic'] = i
    
temp_mosaic = pheno[(pheno.RIC8==1) & (pheno.SSK2==1) & (pheno.SSK1==1)].new_mosaic.min()

pheno.loc[(pheno.RIC8==1) & (pheno.SSK2==1) & (pheno.SSK1==1),'new_mosaic'] = -temp_mosaic

pheno.loc[(pheno.new_mosaic==1),'new_mosaic'] = temp_mosaic
pheno.loc[(pheno.new_mosaic==-temp_mosaic),'new_mosaic'] = 1

assert pheno.new_mosaic.min()>=0

new_mosaic_df = pheno[['SSK1','SSK2','RIC8','new_mosaic']].drop_duplicates().sort_values('new_mosaic').copy()

xlabel = '$SSK1$: %s\n$SSK2$: %s\n$RIC8$: %s'
new_x_labels = []

for i,j in new_mosaic_df.iterrows():
    
    xlab = xlabel%(parental_label[int(j.SSK1)],parental_label[int(j.SSK2)],parental_label[int(j.RIC8)])
    new_x_labels.append(xlab)
    

growth_curves = ['%d_0.0'%d for d in [30,37,39]]

fig,ax= plt.subplots(len(growth_curves),1,figsize=(12,10),sharex=True)
#sns.boxplot(x='new_mosaic',y='H2O2',data=pheno,color='w')
for ix,pt in enumerate(growth_curves):
    plt.sca(ax[ix])
    sns.swarmplot(x='new_mosaic',y=pt, data=pheno);

    for i,j in pheno.groupby('new_mosaic'):
    
        plt.plot(i+.25,j[pt].median(),'^',fillstyle='none',ms=10)

    plt.ylabel('AUC at %s\n with %s amph. B'%(pt.split('_')[0],pt.split('_')[1]),fontsize=12)
    plt.xlabel('')
plt.xlabel('QTG genotypes')
plt.xticks(new_mosaic_df.new_mosaic,new_x_labels,fontsize=10);
plt.savefig('/home/croth/Desktop/temperature_curves_no_drug_amph.B_all_combos.png',
            dpi=100,bbox_inches='tight');

In [ ]:
pheno['new_mosaic'] = -1
for i,a in enumerate(pheno.groupby(['SSK1','SSK2','RIC8'])):
    
    pheno.loc[a[1].index,'new_mosaic'] = i
    
temp_mosaic = pheno[(pheno.RIC8==1) & (pheno.SSK2==1) & (pheno.SSK1==1)].new_mosaic.min()

pheno.loc[(pheno.RIC8==1) & (pheno.SSK2==1) & (pheno.SSK1==1),'new_mosaic'] = -temp_mosaic

pheno.loc[(pheno.new_mosaic==1),'new_mosaic'] = temp_mosaic
pheno.loc[(pheno.new_mosaic==-temp_mosaic),'new_mosaic'] = 1

assert pheno.new_mosaic.min()>=0

new_mosaic_df = pheno[['SSK1','SSK2','RIC8','new_mosaic']].drop_duplicates().sort_values('new_mosaic').copy()

xlabel = '$SSK1$: %s\n$SSK2$: %s\n$RIC8$: %s'
new_x_labels = []

for i,j in new_mosaic_df.iterrows():
    
    xlab = xlabel%(parental_label[int(j.SSK1)],parental_label[int(j.SSK2)],parental_label[int(j.RIC8)])
    new_x_labels.append(xlab)
    

growth_curves = ['%d_0.125'%d for d in [30,37,39]]

fig,ax= plt.subplots(len(growth_curves),1,figsize=(12,10),sharex=True)
#sns.boxplot(x='new_mosaic',y='H2O2',data=pheno,color='w')
for ix,pt in enumerate(growth_curves):
    plt.sca(ax[ix])
    sns.swarmplot(x='new_mosaic',y=pt, data=pheno);

    for i,j in pheno.groupby('new_mosaic'):
    
        plt.plot(i+.25,j[pt].median(),'^',fillstyle='none',ms=10)

    plt.ylabel('AUC at %s\n with %s amph. B'%(pt.split('_')[0],pt.split('_')[1]),fontsize=12)
    plt.xlabel('')
plt.xlabel('QTG genotypes')
plt.xticks(new_mosaic_df.new_mosaic,new_x_labels,fontsize=10);
plt.savefig('/home/croth/Desktop/temperature_curves_0.125_amph.B_all_combos.png',
            dpi=100,bbox_inches='tight');

In [ ]:
import statsmodels.formula.api as smf

In [ ]:
pheno['AUC37'] = pheno['37_0.125']

In [ ]:
pheno['AUC37'].hist()
plt.vlines(pheno['AUC37'].median(),0,20,'b',alpha=0.5)
plt.vlines(pheno['AUC37'].mean(),0,20,'r',alpha=0.5)
plt.vlines(ss.mode(pheno['AUC37'].values,axis=None)[0][0],0,20,'g');

In [ ]:
ss.normaltest

In [ ]:
ss.normaltest(np.sqrt(pheno.AUC37.values))

In [ ]:
pheno['AUC_37'] = np.sqrt(pheno.AUC37.values)

In [ ]:
pheno.AUC_37.hist();

In [ ]:
smf.ols('AUC_37~SSK1+SSK2 + RIC8',data=pheno).fit().summary()

In [ ]:
pheno['AUC39'] = pheno['39_0.0']

In [ ]:
pheno['AUC37amphb'] = pheno['37_0.125']

In [ ]:
pheno[['30_0.125','37_0.125','39_0.125','Melanin']].corr(method='spearman')

In [ ]:
ss.spearmanr(pheno['39_0.125'],pheno['Melanin'])

In [ ]:
sns.regplot(x='Melanin',y='AUC37amphb',data=pheno)

In [ ]:
sns.boxplot(x='SSK1',y='AUC37amphb',hue='SSK2',data=pheno);

In [ ]:
sns.lmplot(x='SSK1',y='AUC37amphb',hue='SSK2',data=pheno,scatter_kws={'alpha':0},legend_out=False);
g = sns.swarmplot(x='SSK1',y='AUC37amphb',hue='SSK2',data=pheno);
g.legend_.remove()

In [ ]:
pheno['AUC39amphb'] = pheno['39_0.125']

In [ ]:
import statsmodels.api as sm

In [ ]:
model= smf.ols('AUC37amphb~SSK1*SSK2',data=pheno).fit()

In [ ]:
anova_table = sm.stats.anova_lm(model,typ=2)

In [ ]:
anova_table

In [ ]:
pheno.columns

In [ ]:
smf.ols('H2O2~RIC8*SSK2',data=pheno).fit().summary()

smf.ols('H2O2~RIC8SSK2',data=pheno).fit().summary()

smf.ols('AUC39AB0125 ~ SSK1 + SSK2*RIC8 - SSK2',data=pheno).fit().summary()

smf.ols('AUC39 ~ RIC8*SSK1*SSK2 - RIC8:SSK1',data=pheno).fit().summary()

smf.ols('Melanin ~ RIC8',data=pheno).fit().summary()

smf.ols('Capsule_diameter_residual~RIC8',data=pheno).fit().summary()

smf.ols('H2O2~RIC8*SSK2*SSK1-SSK2-SSK1:SSK2',data=pheno).fit().summary()

In [ ]:
ss.normaltest(pheno.H2O2.values)

In [ ]:
pheno.to_csv('/home/croth/Desktop/Cdx_phenos.csv.gz')

In [ ]:
fig = plt.figure(figsize=(10,10))
ax = plt.axes(projection='3d')
ms = ['o','^']
for i in [0,1]:
    phe = pheno[(pheno.RIC8==i)]
    ax.scatter3D(phe['39_0.0'], phe['Capsule_diameter_residual'], 
             phe['Melanin'], c=phe['H2O2'], 
             cmap='cividis_r',s=50,alpha=0.5,marker=ms[i]);
ax.view_init(20,20);

ax.set_xlabel('\nAUC at 39C',fontsize=14)
ax.set_ylabel('\nStandardized Capsule Size',fontsize=14)
ax.set_zlabel('\nMelanization',fontsize=14);
plt.savefig('Chromosome_14_phetypes.png',dpi=100,bbox_inches='tight');

In [ ]:
pheno

In [ ]:
pheno[['H2O2','Fludioxonil','30_0.125','37_0.125',
       '37_0.175','39_0.0','39_0.125','Melanin','Capsule_diameter_residual']].corr(method='spearman')

In [ ]:
ss.spearmanr(pheno.H2O2.values,pheno['37_0.125'].values)

In [ ]:
ss.spearmanr(pheno.H2O2.values,pheno['39_0.0'].values)

In [ ]:
for p in ['Fludioxonil','37_0.125','37_0.175','39_0.125','39_0.0','Melanin','Capsule_diameter_residual']:
    
    print(p,ss.spearmanr(pheno.H2O2.values,pheno[p].values))

In [ ]:
h202_pval = geno_df[segs].drop_duplicates().apply(allelic_ANOVA, 
                                                  args = [pheno['39_0.125'].T[segs].values],
                                                 axis=1)

In [ ]:
ss.spearmanr(pheno.H2O2.values,pheno['39_0.125'].values)

In [ ]:
ss.spearmanr(pheno.H2O2.values,pheno['Fludioxonil'].values)

In [ ]:
ssk1 = pd.read_csv('/home/croth/Desktop/Parametric/CNB03090.csv',index_col=0)
ssk2 = pd.read_csv('/home/croth/Desktop/Parametric/CNL05560.csv',index_col=0)
ric8 = pd.read_csv('/home/croth/Desktop/Parametric/CNN01270.csv',index_col=0)

In [ ]:
ssk1gv = geno_df[(geno_df.Chrom=='Chr02') & 
                 (geno_df.Pos>=ssk1.Start.min()) & 
                 (geno_df.Pos<=ssk1.End.max())][segs].mode().dropna()

ssk2gv = geno_df[(geno_df.Chrom=='Chr12') & 
                 (geno_df.Pos>=ssk2.Start.min()) & 
                 (geno_df.Pos<=ssk2.End.max())][segs].mode().dropna()

ric8gv = geno_df[(geno_df.Chrom=='Chr14') & 
                 (geno_df.Pos>=ric8.Start.min()) & 
                 (geno_df.Pos<=ric8.End.max())][segs].mode().dropna()

qtl_gv = pd.concat([ssk1gv,ssk2gv,ric8gv]).T
qtl_gv.columns = ['SSK1','SSK2','RIC8']

In [ ]:
for g in qtl_gv.columns:
    pheno[g] = qtl_gv.T[pheno.index].T[g]

In [ ]:
pheno

In [ ]:
pheno[['Melanin','39_0.0','H2O2','Cell_and_Capsule_diameter','Capsule_diameter','Capsule_diameter_residual']].corr(method='spearman')

In [ ]:
ss.spearmanr(pheno.Melanin.values,pheno.Capsule_diameter_residual)

In [ ]:
sns.regplot(x='39_0.0',y='Capsule_diameter_residual',data=pheno);

In [ ]:
ss.pearsonr(pheno.Capsule_diameter_residual,pheno['39_0.0'])

In [ ]:
ss.spearmanr(pheno.Capsule_diameter_residual,pheno['Melanin'])

In [ ]:
ss.spearmanr(pheno.Capsule_diameter_residual,pheno['39_0.0'])

In [ ]:
ss.spearmanr(pheno.Melanin,pheno['39_0.0'])

In [ ]:
for p in ['Melanin','39_0.0','Capsule_diameter_residual']:
    print(ss.spearmanr(pheno['H2O2'],pheno[p]))

In [ ]:
pheno[['Melanin','39_0.0','Capsule_diameter','Capsule_diameter_residual','H2O2']].corr(method='spearman')

In [ ]:
pheno.to_csv('/home/croth/Desktop/Cdx_phenotypes.csv.gz',index=True)

In [ ]:
sns.scatterplot(x='39_0.0',y='Capsule_diameter_residual',hue='RIC8',data=pheno);

In [ ]:
sns.scatterplot(x='39_0.0',y='Capsule_diameter',hue='RIC8',data=pheno);

In [ ]:
ss.spearmanr(pheno.Capsule_diameter_residual,pheno['39_0.0'])

In [ ]:
ss.spearmanr(pheno.Capsule_diameter,pheno['Melanin'])

In [ ]:
ss.spearmanr(pheno.Capsule_diameter,pheno['Melanin'])

In [ ]:
pheno_z['QTL14'] = pheno.QTL14

In [ ]:
ric8 = np.array([386833, 389698])

In [ ]:
ch14data = pheno[['Melanin','Capsule_diameter_residual','39_0.0','QTL14']].copy()
for p in ch14data.columns.tolist()[:-1]:
    
    ch14data[p] = np.array(myminmax(ch14data[p].values),dtype=float)

In [ ]:
hl,hr = 279819.0, 425700.0
cl,cr = 354579.0, 668746.0
ml,mr = 368955.0, 403449.0

In [ ]:
chr14p = pd.read_csv('./Cdx_chrom_14_QTLs.csv',index_col=0)

In [ ]:
chr14p.head()

In [ ]:
from matplotlib import rc
xl280al = 'XL280'+r'$\alpha$'#+'SS'
xl280_label = 'XL280' +r'$\bf{a}$'
_431_label = '431'+r'$\mathrm{\alpha}$'

In [ ]:
melanin.head()

In [ ]:
ch14data['Mel_QTL'] = melanin['QTL'].T[ch14data.index].T

In [ ]:
ch14data['Melanization QTL Allele'] = [_431_label if a == 1 else xl280_label 
                                         for a in ch14data.Mel_QTL]

In [ ]:
import statsmodels.formula.api as smf

In [ ]:
ch14data.head()

In [ ]:
ch14data.columns = ['Melanin','Capsule_diameter','HTG','QTL14','MELQTL','Melanization QTL Allele']

In [ ]:
ch14data.head()

In [ ]:
ch14data[['Melanin','HTG']].corr().values[0][-1]**2

In [ ]:
HTG_Mel_fit = smf.ols('HTG ~ Melanin',data=ch14data).fit()
htg_mel_rs = np.round(HTG_Mel_fit.rsquared,2)

CELL_Mel_fit = smf.ols('Capsule_diameter ~ Melanin',data=ch14data).fit()
cell_mel_rs = np.round(CELL_Mel_fit.rsquared,2)

In [ ]:
spearman_r = pheno[['Melanin','Capsule_diameter_residual','39_0.0','H2O2']].corr(method='spearman')

In [ ]:
spearman_r

In [ ]:
auc39_cap_cor = np.round(spearman_r[['39_0.0']].T['Capsule_diameter_residual'].min(),2)
auc39_mel_cor = np.round(spearman_r[['39_0.0']].T['Melanin'].min(),2)

In [ ]:
fig,ax = plt.subplots(2,2,figsize=(10,8))
plt.sca(ax[0,1]);plt.axis('off')

plt.sca(ax[0,0])
[ax[0,0].spines[a].set_visible(False) for a in ['top','right']]

kb = 1000
ls = 0.03
yp = -0.05


plt.plot(chr14p.Pos.values/kb,
        myminmax(chr14p.HTG_Pval.values),'.',
         color='lightblue',alpha=0.2,ms=3)#,label=u'39°C QTL')
plt.hlines(yp+0.05,hl/kb,hr/kb,color='lightblue',label=u'39°C QTL\nand 95% CI');
plt.vlines((hl/kb,hr/kb),yp+0.05-ls,yp+0.05+ls,color='lightblue');

plt.plot(chr14p.Pos.values/kb,
         myminmax(chr14p.Residual_Pval.values),
         'r.',alpha=0.2,ms=3)#,label='Capsule\nResidual QTL');
plt.hlines(yp,cl/kb,cr/kb,color='r',label='Cell Diameter Residual QTL\nand 95% CI');
plt.vlines((cl/kb,cr/kb),yp-ls,yp+ls,color='r');

plt.plot(chr14p.Pos.values/kb,
         myminmax(chr14p.Melanin_Pval.values),
         'k.',alpha=0.2,ms=3)#,label='Melanization QTL')
plt.hlines(yp-0.05,ml/kb,mr/kb,color='k',label='Melanization QTL\nand 95% CI');
plt.vlines((ml/kb,mr/kb),yp-0.05-ls,yp-0.05+ls,color='k');
plt.hlines(-0.17,568029/kb,644419/kb,linewidth=15,
           color='blue',label='Centromere');


#plt.plot(chr14p.Pos.values/kb,
#        myminmax(chr14p.H2O2_Pval.values),'.',
#         color='olive',alpha=0.2)#,label=u'39°C QTL')
#plt.hlines(yp-0.3,h2l/kb,h2r/kb,color='olive',label=u'H$_{2}$O$_{2}$ QTL\nand 95% CI');
#plt.vlines((h2l/kb,h2r/kb),yp-0.3-ls,yp-0.3+ls,color='olive');

plt.ylim(-0.15,1.05)
plt.yticks([0,1])
plt.legend(bbox_to_anchor = (2.05,1.15),fontsize=14,frameon=True)

plt.xlabel('Chromosome 14 Coordinates (kb)',fontsize=14);
plt.ylabel('Normalized\n$-\log_{10}\,(p-value)$',fontsize=14);

plt.title('A',fontsize=16,fontweight='bold',x=-.2,y=1)

## ----------------------------------------------------------------------------- ##

plt.sca(ax[1,0])
[ax[1,0].spines[a].set_visible(False) for a in ['top','right']]

#sns.regplot(x='Melanin',y='Capsule_diameter',
#            data=ch14data,scatter_kws={'alpha':0.0},
#            line_kws={'color':'grey','alpha':0.5});
sns.scatterplot(x='HTG',y='Capsule_diameter',data=ch14data,alpha=0.7,
                hue='Melanization QTL Allele',
                hue_order=[xl280_label,_431_label])

plt.legend(bbox_to_anchor=(2.21,1.42),fontsize=14,frameon=True)
plt.xlabel(' ')
plt.ylabel('Normalized\nCapsule Diameter (\u03BCm)',fontsize=14)

plt.ylim(-0.1,1.1)

plt.title('B',fontsize=16,fontweight='bold',x=-.2,y=1)

#plt.text(s='$R^2 = %s$'%cell_mel_rs,fontsize=12,x=0.15,y=1,va='center',ha='center')
plt.text(s='\u03C1 = %s'%auc39_cap_cor,fontsize=12,x=0.75,y=.01,va='center',ha='center')
## ------------------------------------------------------------------------------- ##

plt.sca(ax[1,1])

[ax[1,1].spines[a].set_visible(False) for a in ['top','right']]

#sns.regplot(x='Melanin',y='HTG',data=ch14data,
#            scatter_kws={'alpha':0.0},
#            line_kws={'color':'grey','alpha':0.5});
sns.scatterplot(y='Melanin',x='HTG',data=ch14data,hue='Melanization QTL Allele',
                hue_order=[xl280_label,_431_label],legend=None,alpha=0.7)

plt.ylabel('Normalized Mean Intensity\n(Melanization)',fontsize=14,x=-.2)
plt.xlabel('Normalized AUC at 39°C',fontsize=14,x=-.2)

plt.ylim(-0.1,1.1)
#plt.text(s='$R^2 = %s$'%htg_mel_rs,fontsize=12,x=0.15,y=1,va='center',ha='center')
plt.text(s='\u03C1 = %s'%auc39_mel_cor,fontsize=12,x=0.75,y=.01,va='center',ha='center')

plt.subplots_adjust(wspace=0.3,hspace=0.35)


plt.sca(ax[0,0])
sub1 = fig.add_axes([0.47,.71,0.05,0.01])
plt.xticks([]);plt.yticks([]);
[sub1.spines[a].set_visible(False) for a in ['top','right','left','bottom']]

sub1 = fig.add_axes([0.47,.685,0.05,0.015])
plt.xticks([]);plt.yticks([]);
[sub1.spines[a].set_visible(False) for a in ['top','right','left','bottom']]

#plt.savefig('./Cdx_Chromosome14_QTL_Cell_HTG_Melanization.png',
plt.savefig('/home/croth/QTL_FIGS/SUPP/FIGS/S7_Fig.pdf',
             dpi=300,bbox_inches='tight');

In [ ]:
sns.lmplot(x='39_0.0',y='H2O2',data=pheno,hue='RIC8');

In [ ]:
weirdo = 'SS-B364'

In [ ]:
#ch14data[(ch14data.QTL14==0)].sort_values(by='Melanin')

In [ ]:
combos = pd.DataFrame([np.random.choice([1,1,1,0,0,0],3,replace=False) 
                       for k in range(1000)]
        ).drop_duplicates().sort_values(by=[0,1,2]).reset_index(drop=True)
combos.columns = ['SSK1','SSK2','RIC8']
combos

In [ ]:
combos_ix = [pheno[(pheno.SSK1==j.SSK1) & (pheno.SSK2 ==j.SSK2) & 
                   (pheno.RIC8==j.RIC8)].index for i,j in combos.iterrows()]

In [ ]:
np.sum([len(a) for a in combos_ix])

In [ ]:
[len(a) for a in combos_ix]

In [ ]:
combos_ix_nc = [ [s for s in k if len(s.split('_co')) !=2 and s != weirdo] for k in  combos_ix]

In [ ]:
for_t_temp = [pheno.T[c].T.sort_values(by='H2O2').index[i] for i,c in enumerate(combos_ix_nc[::-1])]

In [ ]:
for_t_temp 

In [ ]:
for_thomas = for_t_temp+['SS-B621','SS-B317','XL280a','SS-A837'] 

In [ ]:
for_t_df = pheno.T[for_thomas].T[['39_0.0','Melanin','Fludioxonil','H2O2','SSK1','SSK2','RIC8']
                     ].sort_values(['SSK1','SSK2','RIC8','H2O2'])
for_t_df.to_csv('~/Cdeneoformans_test_segregants.csv',index=True)

In [ ]:
for_t_df

In [ ]:
plt.plot(pheno.Melanin,pheno.Cell_diameter,'.');
plt.plot(pheno.Melanin,pheno['39_0.0'],'r.');

In [ ]:
pheno_z['Capsule_Cell'] = np.array(ss.zscore(capsule.T[pheno_z.index].T['Cell_diameter'].values),dtype=float)

In [ ]:
np.corrcoef(x = pheno_z['Melanin'].values,y=pheno_z['Capsule_Cell'].values)

In [ ]:
fig,ax = plt.subplots(5,4,figsize=(10,10),sharey=True,sharex=True)

plt.sca(ax[0,0])
plt.title(u'37 °C',fontsize=14,y=.8,x=.2)
sns.scatterplot('30_0.0', '37_0.0',alpha=0.5,hue='QTL14',data = pheno_z,legend=None)

plt.sca(ax[0,1])
plt.title(u'39 °C',fontsize=14,y=.8,x=.2)
sns.scatterplot('37_0.0', '39_0.0',alpha=0.5,hue='QTL14',data = pheno_z,legend=None)
#plt.ylim(-5,85);
#plt.yticks(np.arange(4)*25)

plt.sca(ax[0,2])
plt.plot(0,0,'o',label='XL280',alpha=0.5)
plt.plot(0,0,'o',label='431',alpha=0.5)
plt.plot(0,0,'o',color='white',markersize=20)
plt.legend(title='Chromosome 14 QTL, $RIC8$',
           bbox_to_anchor=(1.2,1),fontsize=14,frameon=False)
for p in ['left','right','top','bottom']:
    ax[0,2].spines[p].set_visible(False)

plt.setp(ax[0,2].get_xticklines(), visible=False)
plt.setp(ax[0,2].get_yticklines(), visible=False)

plt.sca(ax[0,3]);plt.axis('off') 
plt.sca(ax[0,0]);plt.ylabel('AUC (72 hr)',fontsize=14)

    
    
    
plt.sca(ax[1,0])
sns.scatterplot('30_0.0', 'Melanin',alpha=0.5,hue='QTL14',data = pheno_z,legend=None)

plt.sca(ax[1,1])
sns.scatterplot('37_0.0', 'Melanin',alpha=0.5,hue='QTL14',data = pheno_z,legend=None)

plt.sca(ax[1,2])
sns.scatterplot('39_0.0', 'Melanin',alpha=0.5,hue='QTL14',data = pheno_z,legend=None)

plt.sca(ax[1,3]);plt.axis('off');
plt.sca(ax[1,0]);plt.ylabel('Melanization',fontsize=14)




plt.sca(ax[2,0])
#plt.yticks(np.arange(6))
sns.scatterplot('30_0.0', 'H2O2',alpha=0.5,hue='QTL14',data = pheno_z,legend=None)

plt.sca(ax[2,1])
sns.scatterplot('37_0.0', 'H2O2',alpha=0.5,hue='QTL14',data = pheno_z,legend=None)

plt.sca(ax[2,2])
sns.scatterplot('39_0.0', 'H2O2',alpha=0.5,hue='QTL14',data = pheno_z,legend=None)

plt.sca(ax[2,-1])
sns.scatterplot('Melanin','H2O2',alpha=0.5,hue='QTL14',data = pheno_z,legend=None)
plt.sca(ax[2,0])
plt.ylabel(u'H$_{2}$O$_{2}$',fontsize=14)




plt.sca(ax[3,0])
sns.scatterplot('30_0.0', 'Annulus',alpha=0.5,hue='QTL14',data = pheno_z,legend=None)
plt.xlabel(u'AUC at 30 °C',fontsize=14)
#plt.xlim(-5,85)
#plt.ylim(-12,12)

plt.sca(ax[3,1])
sns.scatterplot('37_0.0', 'Annulus',alpha=0.5,hue='QTL14',data = pheno_z,legend=None)
plt.xlabel(u'AUC at 37 °C',fontsize=14)
#plt.xlim(-5,85)

plt.sca(ax[3,2])
sns.scatterplot('39_0.0', 'Annulus',alpha=0.5,hue='QTL14',data = pheno_z,legend=None)
plt.xlabel(u'AUC at 39 °C',fontsize=14)
#plt.xlim(-5,85)

plt.sca(ax[3,-1])
sns.scatterplot('Melanin','Annulus',alpha=0.5,hue='QTL14',data = pheno_z,legend=None)
plt.xlabel('Melanization',fontsize=14)
plt.sca(ax[3,0])
plt.ylabel('Capsule Annulus\nResiduals',fontsize=14)




plt.sca(ax[4,0])
sns.scatterplot('30_0.0', 'Capsule_Cell',alpha=0.5,hue='QTL14',data = pheno_z,legend=None)
plt.xlabel(u'AUC at 30 °C',fontsize=14)
#plt.xlim(-5,85)
#plt.ylim(-12,12)

plt.sca(ax[4,1])
sns.scatterplot('37_0.0', 'Capsule_Cell',alpha=0.5,hue='QTL14',data = pheno_z,legend=None)
plt.xlabel(u'AUC at 37 °C',fontsize=14)
#plt.xlim(-5,85)

plt.sca(ax[4,2])
sns.scatterplot('39_0.0', 'Capsule_Cell',alpha=0.5,hue='QTL14',data = pheno_z,legend=None)
plt.xlabel(u'AUC at 39 °C',fontsize=14)
#plt.xlim(-5,85)

plt.sca(ax[4,-1])
sns.scatterplot('Melanin','Capsule_Cell',alpha=0.5,hue='QTL14',data = pheno_z,legend=None)
plt.xlabel('Melanization',fontsize=14)
plt.sca(ax[4,0])
plt.ylabel('Capsule\nand Cell',fontsize=14)

plt.subplots_adjust(hspace=0.1)
plt.savefig('./Cryptococcus_chromosome14_phenotypes_zscores.png',
            dpi=200,bbox_inches='tight');

In [ ]:
capsule.head()

In [ ]:
sns.swarmplot(x='QTL',y='Capsule',data = capsule);

In [ ]:
pheno_s = pheno.apply(myminmax,axis=0)

In [ ]:
pheno_s.mean()

In [ ]:
pheno_s.std(axis=0,ddof=1)

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(10,10))
pheno_names = pheno.columns.tolist()[:-1]

plt.ylim(-1,len(pheno_names)+1)
plt.xlim(-1,len(pheno_names)+1)
done = []
for i, (l,k) in enumerate(pheno[pheno_names].T.iterrows()):
    for j, (m,z) in enumerate(pheno[pheno_names].T.iterrows()):
        if i == j:
            pass
        elif sorted((i,j)) in done:
            pass
        else:
            r,p = ss.pearsonr(k,z)
            
            if p >= 0.05:
                c = '>0.05'
            elif p <0.05 and p >= 0.01:
                c = '*'
            elif p <0.01 and p > 0.001:
                c = '**'
            else:
                assert p <= 0.001
                c = '***'
    
            plt.text(x=i,y=j,
                     s='%s\n%s'%(np.round(r,3),c),
                     va='center',ha='center')
            done.append(sorted((i,j)))

pheno_new_names = []
for j in pheno_names:
    if len(j.split('_')) == 2:
        new_name = j.split('_')[0]+' °C ' + j.split('_')[1]+u' \u03BCg/ml'
    
    elif j == 'H2O2':
        new_name = 'H$_{2}$O$_{2}$'
    elif j == 'Melanin':
        new_name = 'Melanization'
    elif j == 'Annulus':
        new_name = 'Capsule Annulus\nResidual'
    else:
        new_name = j
    
    pheno_new_names.append(new_name)
    
for i, j in enumerate(pheno_new_names[1:]):
    plt.text(-2,i+1,j,rotation=0,
             va='center',ha='center')
    
for i, j in enumerate(pheno_new_names[:-1]):
    plt.text(i,16,j,rotation=90,
             va='center',ha='center')
    
for p in ['left','right','top','bottom']:
    ax.spines[p].set_visible(False)
    
plt.setp(ax.get_xticklines(), visible=False)
plt.setp(ax.get_yticklines(), visible=False)
plt.xticks([]);plt.yticks([]);

pvals = ['* : 0.01 < p < 0.05',
         '** : 0.001 < p < 0.01',
         '*** : p < 0.001']
for i,j in enumerate(pvals):
    plt.text(8,4-i,s=j,va='center',ha='left',fontsize=14)
plt.savefig('./Corrilation_Phenotypes.png',
            dpi=200,bbox_inches='tight');

In [ ]:
fig,ax = plt.subplots(1,3,figsize=(12,4),sharey=True)
k = '39_0.0'
plt.sca(ax[0])
plt.hist(pheno[k].values)
plt.sca(ax[1])
plt.hist(pheno_mc[k])
plt.sca(ax[2])
plt.hist(pheno_s[k]);

In [ ]:
ps = .90
pca = PCA(ps)
principalComponents = pca.fit_transform(pheno_s)

In [ ]:
principalComponents.shape

In [ ]:
pca.components_.shape

In [ ]:
var_explained = [np.round(a,1) for a in np.round(pca.explained_variance_ratio_,3)*100]
var_explained

In [ ]:
np.sum(var_explained[:3])

In [ ]:
plt.plot(np.arange(1,len(var_explained)+1),np.cumsum(var_explained),'o-');

In [ ]:
principalComponents.T[0].shape

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(10,3),sharex=True,sharey=True)
plt.sca(ax[0])
plt.plot(principalComponents.T[0],principalComponents.T[1],'.');
plt.sca(ax[1])
plt.plot(principalComponents.T[0],principalComponents.T[2],'.');
plt.xlim(-1.5,2);plt.ylim(-1.5,2);

In [ ]:
from sklearn.linear_model import LinearRegression

## Define ftns
def hklod(x,Y):
    x = x.values.reshape(-1,1)
    reg = LinearRegression().fit(x,Y);
    yhat = reg.intercept_+reg.coef_.T*x
    residual = Y - yhat
    resl = np.linalg.det(np.dot(residual.T,residual)) ## Model with one QTL
    res0 = np.linalg.det(np.dot(Y.T,Y)) ## Null model
    return (len(x)/2)*np.log10(res0/resl)

def hkres(x,Y):
    x = x.values.reshape(-1,1)
    reg = LinearRegression().fit(x,Y);
    yhat = reg.intercept_+reg.coef_.T*x
    residual = Y - yhat
    return residual

In [ ]:
len(loci)

In [ ]:
pca_pheno = pd.DataFrame(principalComponents,index=pheno_s.index)
print(pca_pheno.shape)
pca_pheno.columns = ['PC%s'%(i+1) for i in pca_pheno.columns]
pca_pheno.head()

In [ ]:
##hk_lods = [geno[pca_pheno.index].apply(hklod,axis=1,args=[pca_pheno[[K]]]) 
##          for K in pca_pheno.columns] 

perms = 1000
thresholds = []
for i in range(perms):
    pvals = [geno[pca_pheno.index].apply(association_logPval,axis=1,
            args=[pca_pheno[K][np.random.permutation(pca_pheno.index.values)].values]) 
               for K in pca_pheno.columns]
    
    maxp = np.max(pvals,axis=0).max()
    meanp = np.mean(pvals,axis=0).max()
    thresholds.append((maxp,meanp))
thres = pd.DataFrame(thresholds,columns=['Max','Mean'])
thres.to_csv('~/PCA_phenotypes_thresholds.csv',index=False)

In [ ]:
thres = pd.read_csv('/home/croth/PCA_phenotypes_thresholds.csv')

In [ ]:
thres.head()

In [ ]:
mean_95 = np.percentile(thres.Mean.values,95)
mean_99 = np.percentile(thres.Mean.values,99)

In [ ]:
max_95 = np.percentile(thres.Max.values,95)
max_99 = np.percentile(thres.Max.values,99)

In [ ]:
pvals = [geno[pca_pheno.index].apply(association_logPval,axis=1,
        args=[pca_pheno[K].values]) for K in pca_pheno.columns]

pval_df = pd.DataFrame(pvals,
            columns=geno.index,
            index=pca_pheno.columns).T

pval_df[['Chrom','Pos']] = geno_df[['Chrom','Pos']].T[loci].T

In [ ]:
##ylabel = 'LOD'
ylabel = r'-log$_{10}$(p-value)'

In [ ]:
fig,ax = plt.subplots(2,1,figsize=(12,5),sharex=True)
plt.sca(ax[0]);plt.ylabel(ylabel)
[plt.hlines(a,0,19*10**6,linestyle='--',alpha=0.5) for a in [max_95,max_99]]
for i,c in enumerate(chrlist):
    
    pt = pval_df[(pval_df.Chrom==c)]
    
    plt.plot(pt.Pos+cumpos[i],pt[pca_pheno.columns].max(axis=1),'.')


plt.sca(ax[1]);plt.ylabel(ylabel);plt.xlabel('Chromosome')
[plt.hlines(a,0,19*10**6,linestyle='--',alpha=0.5) for a in [mean_95,mean_99]]
for i,c in enumerate(chrlist):
    
    pt = pval_df[(pval_df.Chrom==c)]
    
    plt.plot(pt.Pos+cumpos[i],pt[pca_pheno.columns].mean(axis=1),'.')
plt.xticks(chrom_midpts,np.arange(1,15));


In [ ]:
pheno_s['Ch02PC1'] = geno.T[pval_df[(pval_df.Chrom=='Chr02')]['PC1'].idxmax()]
pheno_s['Ch14PC1'] = geno.T[pval_df[(pval_df.Chrom=='Chr14')]['PC1'].idxmax()]
pheno_s['Ch12PC5'] = geno.T[pval_df[(pval_df.Chrom=='Chr12')]['PC5'].idxmax()]
pheno_s['Ch02PC7'] = geno.T[pval_df[(pval_df.Chrom=='Chr02')]['PC7'].idxmax()]

In [ ]:
fig,ax = plt.subplots(2,8,figsize=(14,5))

for i,p in enumerate(pheno_mc.columns):
    
    plt.sca(ax.ravel()[i])
    
    sns.regplot(x='Ch02PC7',y=p,data=pheno_s,x_jitter=0.2);
plt.subplots_adjust(hspace=0.7,wspace=0.7)

In [ ]:
fig,ax = plt.subplots(pca_pheno.shape[1],pca_pheno.shape[1],
                      figsize=(10,10),sharex=True)

for i, p1 in enumerate(pca_pheno.columns):
    for j, p2 in enumerate(pca_pheno.columns):
        plt.sca(ax[i,j])
        if i == j:
            plt.hist(pca_pheno[p1],alpha=0.5)
        else:
            plt.plot(pca_pheno[p1].values,
                     pca_pheno[p2].values,'.',alpha=0.5)
            plt.xlim(-5,9);plt.ylim(-5,9);
plt.subplots_adjust(hspace=0.5,wspace=0.5)

fig,ax = plt.subplots(pheno_s.shape[1],pheno_s.shape[1],
                      figsize=(10,10),sharex=True)

for i, p1 in enumerate(pheno_s.columns):
    for j, p2 in enumerate(pheno_s.columns):
        plt.sca(ax[i,j])
        if i == j:
            plt.hist(pheno_s[p1],alpha=0.5)
        else:
            plt.plot(pheno_s[p1].values,pheno_s[p2].values,'.',alpha=0.5)

In [ ]:
fig,ax = plt.subplots(len(pca_pheno.columns),1,
                          figsize=(12,12),sharex=True,sharey=True)
for j,k in enumerate(pca_pheno.columns):
    plt.sca(ax.ravel()[j]);
    plt.ylabel(ylabel);
    plt.xlabel('Chromosome' 
            if j == len(pca_pheno.columns) else None)
    
    plt.title('%s %s'%(k,var_explained[j]),y=.65)
    for i,c in enumerate(chrlist):
    
        pt = pval_df[(pval_df.Chrom==c)]
    
        plt.plot(pt.Pos+cumpos[i],pt[k],'.')

    plt.xticks(chrom_midpts,np.arange(1,15));

In [ ]:

ch2qtlpc7 = pd.DataFrame([geno.T[ch2pc7max][pca_pheno.index],
                          pca_pheno['PC7']],
                         index=['QTL2','PC7']).T

sns.lmplot(x='QTL2',y='PC7',data=ch2qtlpc7,legend=None);
plt.xlabel('Chromosome 2 QTL')
plt.xticks([0,1]);

In [ ]:





ch12qtlpc5 = pd.DataFrame([geno.T[ch12pc5max][pca_pheno.index],
                          pca_pheno['PC5']],
                         index=['QTL12','PC5']).T

sns.lmplot(x='QTL12',y='PC5',data=ch12qtlpc5,legend=None);
#plt.legend(title='Chromosome 14 QTL');
plt.xlabel('Chromosome 12 QTL')
plt.xticks([0,1]);

In [ ]:


ch2qtlpc1 = pd.DataFrame([geno.T[ch2pc1max][pca_pheno.index],
                          geno.T[ch14pc1max][pca_pheno.index],
                          pca_pheno['PC1']],
                         index=['QTL2','QTL14','PC1']).T

sns.lmplot(x='QTL2',y='PC1',data=ch2qtlpc1,hue='QTL14',legend=None);
plt.legend(title='Chromosome 14 QTL');
plt.xlabel('Chromosome 2 QTL')
plt.xticks([0,1]);

In [ ]:
pca_pheno

In [ ]:
pilot_path = './CRNE_Progeny_QTL_Phenotyping_All_20150226_Renamed.xlsx'
pilot = pd.read_excel(pilot_path)
pilot.columns = ['strain','ypd30','ypd37','ypd40','ypd41','ypd42',
                    'Ldopa','Ms','Fluc','Amph','Ph']
pilot.index= pilot.strain
pilot.drop('strain',axis=1,inplace=True)
pilot.head()

In [ ]:
len(segs)

In [ ]:
fluc_segs = [s for s in segs if s in pilot.index.tolist()];len(fluc_segs)

In [ ]:
miss_fluc = [s for s in segs if s not in pilot.index.tolist()]
miss_fluc

In [ ]:
pval_df['Fluc'] = geno[fluc_segs].apply(
                    association_logPval,axis=1,
                    args=[pilot['Fluc'][fluc_segs].values])

In [ ]:
pval_df.head()

fluc_perms = []
while len(fluc_perms) < 1000:
    flucperm = geno[fluc_segs].apply(association_logPval,axis=1,
        args=[pilot['Fluc'][np.random.permutation(fluc_segs)].values]).max()
    fluc_perms.append(flucperm)
fluc_permdf = pd.DataFrame(fluc_perms)
fluc_permdf.to_csv('./Fluconazole_permutations.csv')

In [ ]:
fluc_thres = pd.read_csv('./Fluconazole_permutations.csv',index_col=0)

In [ ]:
fluc_95 = np.percentile(fluc_thres.values,95)
fluc_99 = np.percentile(fluc_thres.values,99)

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(10,2))
for i,c in enumerate(chrlist):
    tp = pval_df[(pval_df.Chrom==c)]
    plt.plot(tp.Pos.values+cumpos[i],tp.Fluc.values,'.')
plt.hlines(fluc_99,0,19*10**6,linestyle='--',alpha=0.5);
plt.ylabel('-log$_{10}$(p-value)',fontsize=14)
plt.xlabel('Chromosome')
plt.xticks(chrom_midpts,np.arange(1,15));

In [ ]:
ch2_geno = geno_df[(geno_df.Chrom=='Chr02')][fluc_segs]
ch2_geno = ch2_geno.T[[l for l in loci if l in ch2_geno.index]].T

In [ ]:
ch2_geno.head()

In [ ]:
fluc_boots = []
while len(fluc_boots) < 250:
    
    ran_segs = np.random.choice(fluc_segs,len(fluc_segs))
    
    fluc_boot = ch2_geno[ran_segs].apply(association_logPval,axis=1,
        args=[pilot['Fluc'][np.random.permutation(ran_segs)].values])
    
    fluc_boots.append(fluc_boot.idxmax())

In [ ]:
ch2_pos = geno_df.T[fluc_boots].T[['Pos','Chrom','Block']].sort_values('Pos')

In [ ]:
chr2 = pval_df[(pval_df.Chrom=='Chr02')]

In [ ]:
plt.plot(chr2.Pos/1000,chr2.Fluc,'.')
plt.vlines(np.percentile(ch2_pos.Pos,5)/1000,-1,0,color='black')
plt.vlines(np.percentile(ch2_pos.Pos,95)/1000,-1,0,color='black')
plt.hlines(-0.5,np.percentile(ch2_pos.Pos,5)/1000,
           np.percentile(ch2_pos.Pos,95)/1000,color='k');
plt.yticks(np.arange(6));
plt.text(x=clens[1]/2000,y=-.8,s='95% CI',va='center',ha='center')
plt.ylabel('-log$_{10}$(p-value)',fontsize=14)
plt.xlabel('Chromosome 2 Coordinates (kb)',fontsize=14);

In [ ]:
plt.plot(chr2.Pos.values/1000,chr2['PC7'].values/chr2['PC7'].max(),
         '-.',label='PC7',alpha=0.8)
plt.plot(chr2.Pos.values/1000,chr2['Fluc'].values/chr2['Fluc'].max(),
         '-.',label='Fluconazole',alpha=0.8);
plt.legend(fontsize=14)
plt.ylabel('Normalized\n-log$_{10}$(p-value)',fontsize=14)
plt.xlabel('Chromosome 2 Coordinates (kb)',fontsize=14);